In [1]:
import pandas as pd
import networkx as nx
%pylab inline

from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [2]:
tables = pd.read_csv('./data/Tables.csv')

csvs = tables['Table'].unique()
index_cols = [csv.replace('ies', 'y') if csv.endswith('ies') else csv.rstrip('s') for csv in csvs]
zipped = dict(zip(csvs, index_cols))

entities = {
    csv: pd.read_csv('./data/{}.csv'.format(csv), index_col=index_col).replace({
        np.nan: None
    }) for csv, index_col in zipped.items()
}

In [3]:
# mismatch repr.
entities['Resources'].loc['Uranium', 'Enables'] = 'Nuclear Device, Thermonuclear Device, Nuclear Submarine, Modern Armor'
entities['Units'].loc['Rough Rider']['Unique to'] = 'Teddy Roosevelt'

# typos
entities['Policies'].loc[['Ilkum', 'Agoge'], 'Civic'] = 'Craftsmanship'
entities['Units'].loc['Maryannu Chariot Archer'][['Unique to', 'Replaces']] = pd.Series(['Egyptian', 'Heavy Chariot'])

In [4]:
cost_cols = dict(zip(['Technologies', 'Wonders'], ['Science', 'Production']))
for csv, df in entities.items():
    
    # make table-specific Type attribute 
    if 'Type' in df.columns:
        df.rename(columns={
            'Type': df.index.name + ' Type'
        }, inplace=True)
        
    # add node Type attribute
    df['Type'] = df.index.name
    tables = tables.append(pd.DataFrame({
        'Table': csv,
        'Columns': 'Type',
        'Repr.': 'Attribute'
    }, index=range(1)), sort=False, ignore_index=True)
    
    # change Cost columns to units
    if csv in cost_cols.keys():
        df.rename(columns={
            'Cost': cost_cols[csv]
        }, inplace=True)
        tables.loc[(tables['Table'] == csv) & (tables['Columns'] == 'Cost'), 'Columns'] = cost_cols[csv]

In [5]:
G = nx.DiGraph()

attrs_cols = (tables[tables['Repr.'] == 'Attribute']
              .groupby('Table')
              ['Columns'].apply(list)
              .items())

for csv, cols in attrs_cols:
    cost_cols = [col for col in cols if col in ['Cost', 'Stats']]
    cols = [col for col in cols if col not in cost_cols]
    
    nodes = [(node, {
        attr: val for attr, val in attrs.items() if val
    }) for node, attrs in entities[csv][cols].to_dict('index').items()]
    G.add_nodes_from(nodes)
    
    nx.set_node_attributes(G, {
        n: {
            'Type': zipped[csv]
        } for n in [node[0] for node in nodes]
    })
    
    if cost_cols:
        costs = {
            node: {
                c.split(': ')[0]: int(c.split(': ')[1]) for cs in costs.values() if cs for c in cs.split(', ')
            } for node, costs in entities[csv][cost_cols].to_dict('index').items()
        }
        nx.set_node_attributes(G, costs)

In [6]:
edge_cols = (tables[(tables['Repr.'] == 'Edge') & (tables['Notes'] != 'Redundancy')]
             .groupby(['Table', 'Edge Type'])
             ['Columns'].apply(list)
             .items())

for (csv, edge_key), cols in edge_cols:
    for col in cols:
        edges = list(entities[csv][col]
                     .str.split(', ', expand=True)
                     .stack()
                     .reset_index(level=-1, drop=True)
                     .items())
        
        edges, edge_type = ([(v, u) for u, v in edges], edge_key.split(', ')[0]) if 'reverse' in edge_key else (edges, edge_key)
        G.add_edges_from(edges, Type=edge_type)
        
        if col == 'Unique to':
            specs = {
                node: {
                    'Specificity': 'Civilization'
                } for edge in edges for node in edge
            }
            nx.set_node_attributes(G, specs)

In [7]:
edge_cols = (tables[tables['Repr.'] == 'Node, Edge']
         .groupby(['Table', 'Edge Type'])
         ['Columns'].apply(list)
         .items())
casus = [
    'Holy War', 'Liberation', 'Reconquest', 'Protectorate', 'War of Retribution',
    'War of Territorial Expansion', 'War of Territorial Expansion'
]

for (csv, edge_key), cols in edge_cols:
    for col in cols:
        edges = list(entities[csv][col]
                     .str.split(', ', expand=True)
                     .stack()
                     .str.replace('Casus Belli: ', '')
                     .reset_index(level=-1, drop=True)
                     .items())
        edges, edge_type = ([(v, u) for u, v in edges], edge_key.split(', ')[0]) if 'reverse' in edge_key else (edges, edge_key)
        G.add_edges_from(edges, Type=edge_type)
        
        types = {
            dip.replace('Casus Belli: ', ''): {
                'Type': 'Casus Belli'
            } if dip.replace('Casus Belli: ', '') in casus else {
                'Type': 'Diplomacy'
            } for _, dip in edges
        } if col == 'Diplomacies' else {
            leader: {
                'Type': 'Leader(s)'
            } for _, leader in edges 
        } if col == 'Leader(s)' else {
            city: {
                'Type': 'City-state'
            } for city, _ in edges if city not in entities['Civilizations'].index
        }
        nx.set_node_attributes(G, types)
        
        if col == 'Unique to':
            specs = {
                node: {
                    'Specificity': 'Civilization'
                } for edge in edges for node in edge
            }
            nx.set_node_attributes(G, specs)

In [8]:
nuc_attrs = {
    nuc: {
        'Type': 'Atomic Weapon'
    } for nuc in ['Nuclear Device', 'Thermonuclear Device']
}
nx.set_node_attributes(G, nuc_attrs)

In [9]:
builds = [('Builder', improvement, {
    'Type': 'Builds'
}) for improvement in entities['Improvements'].index]
G.add_edges_from(builds)

In [10]:
# remove edge redundancy from nodes with "replaces" edge
replaces = [u for (u, v), p in nx.get_edge_attributes(G, 'Type').items() if p == 'Replaces']
removes = [(u, v) for (u, v), p in nx.get_edge_attributes(G, 'Type').items() if p != 'Replaces' and u in replaces and 'Carnival' not in u]
G.remove_edges_from(removes)

In [11]:
GS_nodes = ['Force Modernization', 'Buttress', 'Courser']
isolates = list(nx.isolates(G))
G.remove_nodes_from(GS_nodes + isolates)

In [13]:
eurekas = nx.get_node_attributes(G, 'Eureka')
insps = nx.get_node_attributes(G, 'Inspiration')

In [14]:
eureka_nodes = [
    '', '', '', '', 'Slinger', 'Quarry', '', '', '', 'Trader', 'Pasture', 'Iron',
    'Galley', '', 'Water Mill', 'Ancient Walls', 'Spearman', 'Mine', 'Feudalism', 'Archer',
    '', 'Aqueduct', '', 'Harbor', 'Lumber Mill', 'Guilds', 'Armory', 'University', 'Musketman',
    'University', 'Crossbowman', 'Bombard', 'Workshop', 'The Enlightenment', 'Fort', 'Knight',
    'Shipyard', 'Neighborhood', 'Bank', 'Niter', '', 'Musketman', ['Coal, Ironclad'], 'Privateer', '',
    'Research Agreement', 'Artifacts', 'Biplane', 'Spy', 'Power Plant', 'Airstrip', 'Oil Well',
    '', 'Spy', 'Aerodrome', 'Broadcast Center', 'Spy', '', 'Spy', 'Tank', 'Spy',
    'Globalization', 'Spy', 'Aluminum'
]

In [15]:
insp_nodes = [
    'Stock Exchange', '', '', 'Factory', 'Nuclear Fission', 'Astronomy', 'Neighborhood', '', '', '', 'Alliances',
    'Temple', '', '', 'Caravel', 'Farm', '', 'Construction', 'Airport', 'Market', '', 'Radio', 'Trader', '', '', '',
    'Encampment', '', '', '', 'Archaeological Museum', 'Quadrireme', 'Research Lab', 'Art Museum', '',
    'Entertainment Complex', '', 'Campus', '', 'Field Cannon', 'Telecommunications', 'Spaceport', '', 'Sewer', '', '',
    'Military Academy', ''
]

In [16]:
eureka_boosts = []
for (v, boost), u in zip(eurekas.items(), eureka_nodes):
    if u:
        if not isinstance(u, list):
            eureka_boosts.append((u, v, {
                'Type': 'Boosts',
                'Boost': boost
            }))
        else:
            for n in u[0].split(', '):
                eureka_boosts.append((n, v, {
                    'Type': 'Boosts',
                    'Boost': boost
                }))
G.add_edges_from(eureka_boosts)
                
                
insp_boosts = [(u, v, {
    'Type': 'Boosts',
    'Boost': boost
}) for (v, boost), u in zip(insps.items(), insp_nodes) if u]
G.add_edges_from(insp_boosts)

In [17]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 627
Number of edges: 1125
Average in degree:   1.7943
Average out degree:   1.7943


In [19]:
Counter(nx.get_edge_attributes(G, 'Type').values())

Counter({'Boosts': 77,
         'Builds': 34,
         'Harvests': 11,
         'Obsoletes': 87,
         'Replaces': 43,
         'Reveals': 8,
         'Unlocks': 802,
         'Upgrades': 63})

In [26]:
# nx.write_graphml(G, 'civ.graphml')